In [1]:
import os, sys
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from skimage import io

# out = widgets.Output()

from lib.labelling import *
from sidecar import Sidecar

In [2]:
plt.close('all')

obj = image_segmenter('images', ['mother','daughter'])



sc = Sidecar(title='Segmentation area')
with sc:
    display(obj)
